In [1]:
# Cell 1: Install required packages
!pip install streamlit PyPDF2 python-docx docx2txt fpdf reportlab pyngrok -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.7 MB/s eta 0:00:00


In [10]:
# Cell 2
# Write in a file app.py
%%writefile app.py

import streamlit as st
from PyPDF2 import PdfReader
import docx2txt
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

# Import io
io = __import__("io")

# Function to read the contents of the file
def read_file(file):
    if file.name.endswith(".pdf"):
        pdf_reader = PdfReader(file)
        text = []
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                lines = page_text.splitlines()
                text.extend(lines)
        return "\n".join(text)
    elif file.name.endswith(".doc") or file.name.endswith(".docx"):
        return docx2txt.process(file)
    else:
        return "Error: Unsupported file type"

# Function to process and merge contents into a PDF
def process_files(file1, file2):
    resume = read_file(file1)
    job_description = read_file(file2)
    enhanced_content = resume + "\n\n" + job_description

    # Creating a PDF using reportlab
    pdf_output = io.BytesIO()
    doc = SimpleDocTemplate(pdf_output, pagesize=letter)
    styles = getSampleStyleSheet()

    body_style = styles["BodyText"]
    body_style.leading = 14  # Adjust line spacing
    elements = []

    # Process lines as paragraphs to keep structure
    for paragraph in enhanced_content.split("\n\n"):
        lines = paragraph.split("\n")
        for line in lines:
            elements.append(Paragraph(line, body_style))
        elements.append(Spacer(1, 12))  # Space between paragraphs

    doc.build(elements)
    pdf_output.seek(0)
    return pdf_output

# Streamlit Interface
st.title("ResumeTune")

file1 = st.file_uploader("Upload File 1", type=["pdf", "doc", "docx"])
file2 = st.file_uploader("Upload File 2", type=["pdf", "doc", "docx"])

if st.button("Enhance your Resume"):
    if file1 is not None and file2 is not None:
        merged_pdf = process_files(file1, file2)
        st.download_button(
            label="Download Enhanced PDF",
            data=merged_pdf,
            file_name="enhanced_resume.pdf",
            mime="application/pdf",
        )
    else:
        st.error("Please upload two files to merge.")

Overwriting app.py


In [11]:
from pyngrok import ngrok, conf, process
import os
import time

# Ensure previous sessions are killed
!pkill streamlit

# Start the Streamlit app
port = 8501
os.system(f"streamlit run app.py --server.port {port} &")

# Wait a bit for Streamlit to start
time.sleep(5)

# Configure ngrok with your authtoken
# Replace <your_authtoken> with your actual authtoken
conf.get_default().auth_token = "2oA2P8ECRxHzy0VQamBIrcr6w04_5nvMNVm9QnXijSFTaqxoq"

ngrok.kill()

try:
    # Open ngrok tunnel and print the public URL
    public_url = ngrok.connect(port)
    print("Streamlit App URL:", public_url)
except Exception as e:
    print(f"An error occurred during ngrok connection: {e}")

Streamlit App URL: NgrokTunnel: "https://11d4-34-81-198-62.ngrok-free.app" -> "http://localhost:8501"
